# 问题转化为向量空间的数学运算

In [ ]:
import numpy as np

def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    
    return sim

import tensorflow_hub as hub


embed = hub.load("model/nnlm-zh-dim50")




In [ ]:
embed(["学生","同学"])
# zhou'm

In [ ]:
def embeddings_cos_sim(ab):
    embeddings = embed(ab)
    B=embeddings.numpy()[1]
    A=embeddings.numpy()[0]
    print(ab,cos_sim(A,B))

    
#embeddings_cos_sim(["猫","狗"])


#embeddings_cos_sim(["狗","桌子"])

embeddings_cos_sim(["建筑设计","空间设计"])

# embeddings_cos_sim(["设计","数学"])
# embeddings_cos_sim(["设计","编程"])


# 特征工程、机器学习

In [ ]:
from sklearn import datasets

import matplotlib.pyplot as plt

import numpy as np

from sklearn import tree

# Iris数据集是常用的分类实验数据集，
# 由Fisher, 1936收集整理。Iris也称鸢尾花卉数据集，
# 是一类多重变量分析的数据集。数据集包含150个数据集，
# 分为3类，每类50个数据，每个数据包含4个属性。
# 可通过花萼长度，花萼宽度，花瓣长度，花瓣宽度4个属性预测鸢尾花卉属于（Setosa，Versicolour，Virginica）三个种类中的哪一类。

#载入数据集
iris = datasets.load_iris()
#print(iris)
iris_data=iris['data']
#print(iris_data[0])

iris_label=iris['target']
#print(iris_label[0])
iris_target_name=iris['target_names']
print(iris_target_name)

# X=np.array(iris_data)
# Y=np.array(iris_label)

# print(X[0],iris_target_name[0])

# # #训练,决策树
# model=tree.DecisionTreeClassifier(max_depth=3)
# # 开始训练
# model.fit(X,Y)
 
# #这里预测当前输入的值的所属分类
# print('预测类别是',iris_target_name[clf.predict([[5,3,1,0.1]])[0]])



In [ ]:
k=[1,2,3,4]
k[0]

In [ ]:
#print(X.shape , np.array([X[0]]).shape)

model.predict([ [1,3,0.5,6] ] )[0]
print('预测类别是',iris_target_name[0])

# 深度学习-表示学习，万物皆向量

In [ ]:
import tensorflow_hub as hub
embed = hub.load("model/nnlm-zh-dim50")
embeddings = embed(["猫","狗"])

In [ ]:
embeddings = embed(["狗","桌子"])

In [ ]:
embeddings = embed(["男人","女人"])

In [ ]:
B=embeddings.numpy()[1]
A=embeddings.numpy()[0]

In [ ]:
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim
cos_sim(A,B)

In [ ]:
#欧氏距离和余弦相似度
def dist_sim(vector_a, vector_b):
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    dist = np.linalg.norm(vector_a - vector_b)  
    sim = 1.0 / (1.0 + dist) #归一化  
    return sim

In [ ]:
dist_sim(A,B)

# 机器学习 分类
## 找出群体中的KOL
### 对比欧式距离与余弦相似度

In [ ]:
#加载数据
import pandas as pd
df = pd.read_csv("data/students.csv")

In [ ]:
#查看下数据的前几条
df.head()

In [ ]:
#取需要的列
student=df.loc[1,['Name','Email','School','Major','grade','Interest','Interest level','Code']].values.tolist()

In [ ]:
#加载 预训练模型
import tensorflow_hub as hub
embed = hub.load("model/nnlm-zh-dim50")

In [ ]:
# 测试下
embeddings = embed(["".join(student)])[0]
embeddings.numpy()

In [ ]:
#把特征转成 稠密向量
students=[]
for i in range(len(df)):
    #print(i)
    student=df.loc[i,['Email','School','Major','grade','Interest','Interest level','Code']].values.tolist()
    students.append(embed(["".join(student)])[0].numpy())
students=np.array(students)

In [ ]:
#每位同学的稠密向量
students

In [ ]:
#使用scikit learn的余弦相似度计算方法
from sklearn.metrics.pairwise import cosine_similarity
sim=cosine_similarity(students)
#查看下第一位与第二位同学的相似度
sim[0][1]

In [ ]:
#为每位同学计算与他最相似的一位同学，只取相似度大于0.6的第一位同学
count_students={}
for i in range(len(students)):
    others=[]
    for j in range(len(students)):
        if i!=j:
            others.append({
                "index":j,
                "score":sim[i][j]
            })
    others=sorted(others, key=lambda x:x["score"],reverse=True)
    if others[0]['score']>0.6:
        print(df.loc[i,'Name'],df.loc[others[0]['index'],'Name'],others[0]['score']) 
        if not df.loc[others[0]['index'],'Name'] in count_students:
            count_students[df.loc[others[0]['index'],'Name']]=0
        count_students[df.loc[others[0]['index'],'Name']]+=others[0]['score']

In [ ]:
# 不太可能是kol的同学（与其他同学相似性较低)
for i in range(len(students)):
    if not df.loc[i,'Name'] in count_students:
        print(df.loc[i,'Name'])

In [ ]:
# 最有可能是kol的同学
sorted(count_students.items(), key=lambda x:x[1],reverse=True)

# 聚类算法
### 聚类班级学生

In [ ]:
#引入库，并测试
from sklearn.cluster import KMeans,DBSCAN,Birch
import numpy as np

X = np.array([[1,2, 2], [1,2, 4], [1,2, 0],[4, 2,2], [4,2, 4], [4,1, 0]])

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

kmeans.labels_ #输出原始数据的聚类后的标签值


In [ ]:
#设置数量
model = KMeans(n_clusters =2)

In [ ]:
#训练
model.fit(students)

In [ ]:
for i in range(len(model.labels_)):
    if model.labels_[i]==0:
        print(df.loc[i,'Name'])
    

In [ ]:
#换一种
model=DBSCAN(eps=0.11, min_samples=2).fit(students)
print(model.labels_)

In [ ]:
#换一种
model = Birch(n_clusters=2)
model.fit(students)
print(model.labels_)

In [ ]:
# 可视化查看模型学习到的分类
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2)
decomposition_data = tsne.fit_transform(students)

x = []
y = []
 
for i in decomposition_data:
    x.append(i[0])
    y.append(i[1])
    
plt.figure(figsize=(10, 10)) 
ax = plt.axes() 
plt.scatter(x, y, c=model.labels_, marker="x") 
plt.xticks(()) 
plt.yticks(()) 
plt.show()

# 深度学习 hello world
### 手写数字分类

In [ ]:
%load_ext tensorboard

#引入相关库
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
import datetime

In [ ]:
# 下载mnist数据集
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# 数据集维度
print(x_test.shape,y_test.shape)

In [ ]:
#查看下数据集
plt.imshow(x_test[1882])
plt.show()

In [ ]:
#模型定义
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10, activation='relu'),
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#训练开始
model.fit(x_train, y_train, epochs=5)
#评估模型
model.evaluate(x_test,  y_test, verbose=2)

# 颜色分类 v1.0

### 使用 Pandas 处理数据

In [ ]:
#使用 Pandas 创建一个 dataframe
import pandas as pd
dataframe=pd.read_json("data/colorData.json",orient="records")

In [ ]:
#预览下前面几条数据
dataframe.head()

In [ ]:
#查看下数据类型
dataframe.dtypes

In [ ]:
#label需要转成int
dataframe['label'] = pd.Categorical(dataframe['label'])
dataframe['label'] = dataframe.label.cat.codes

In [ ]:
# 获取标签名称
#code--label
def get_label_name(label=0):
    labels=pd.Categorical(['brown-ish','blue-ish', 'green-ish', 'grey-ish', 'orange-ish',
           'pink-ish', 'purple-ish', 'red-ish', 'yellow-ish'])
    index=labels.codes.tolist().index(label)
    return labels.categories.tolist()[index]

In [ ]:
# 测试下
get_label_name(3)

In [ ]:
#查看标签分布
dataframe.loc[:,'label'].value_counts()

In [ ]:
#准备训练数据
dataframe=dataframe[['r','g','b','label']]

### 制作数据集

In [ ]:
#分割成训练集、验证集、测试集
from sklearn.model_selection import train_test_split

df=dataframe.copy()

train, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
# 一种从 Pandas Dataframe 创建 tf.data 数据集的实用程序方法（utility method）
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    x = dataframe.copy()
    x=x.astype('float64')
    y = x.pop('label')
    ds = tf.data.Dataset.from_tensor_slices((x.values, y.values))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
#参数

#Batch Size：一次训练所选取的样本数。
BATCH_SIZE=64
#失活率(Dropout Rate) 每层中丢弃的神经元占整层神经元的比率
DROPOUT_RATE=0.1045
#轮次，整个输入数据的单次前向和反向传递
EPOCHS=100

log_dir = "logs/fit/DROPOUT_RATE_" + str(DROPOUT_RATE)+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



In [ ]:
train_ds=df_to_dataset(train,batch_size=BATCH_SIZE)
val_ds=df_to_dataset(val,batch_size=BATCH_SIZE)
test_ds=df_to_dataset(test,batch_size=BATCH_SIZE)

In [ ]:
for f in train_ds.take(1):
    print(f)

### 模型

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(12, input_shape=(3,),activation='softplus'),
  tf.keras.layers.Dense(48, activation='relu'),
  tf.keras.layers.Dropout(DROPOUT_RATE),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(DROPOUT_RATE),
  tf.keras.layers.Dense(48, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#optimizer=tf.keras.optimizers.SGD(learning_rate=0.25)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=EPOCHS,
             callbacks=[tensorboard_callback])



In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
test_loss, test_acc = model.evaluate(test_ds, verbose=2)
print('\nTest accuracy:', test_acc)

### 预测

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
import numpy as np
predictions = probability_model.predict(np.array([[110,2,25]]))

In [ ]:
predictions[0]

In [ ]:
get_label_name(np.argmax(predictions[0]))

In [ ]:
test

# RGB转HSV
### rgb->hsv

In [ ]:
import cv2
# opencv
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#创建一张黑色的图片
img = np.zeros((28,32), np.float32)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
print(img.shape)
plt.imshow(img)
plt.show()

In [ ]:
rgb_to_hsv([110,2,25])

In [ ]:
'''
HSV颜色空间规定:H范围0~360,S范围0~1,V范围0~1 

PS中的HSV范围，H是0-360，S是0-1，V（B）是0-1

opencv中的HSV范围，H是0-180，S是0-255，V是0-255

'''

# h:0-360 , s:0-255, v:0-255
# r:0-255, g:0-255, b:0-255
def rgb_to_hsv(rgb=[]):
    img=np.array([[rgb]],np.uint8)
    #print(img)
    #print(img)
    plt.imshow(img)
    plt.show()
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    img_hsv[0][0]*=2
    #print(img_hsv)
    return img_hsv


def hsv_to_rgb(hsv=[]):
    hsv[0]/=2
    img_hsv=np.array([[hsv]],np.uint8)
    img_rgb = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
    plt.imshow(img_rgb)
    plt.show()
    return img_rgb

#rgb_to_hsv([250,0,0])
#hsv_to_rgb([360,255,255])

# 从h=0 开始旋转，每18°取一组颜色，作为配色方案
for i in range(0,360,18):
    #print(i)
    a=hsv_to_rgb([i,255,255])
    b=hsv_to_rgb([i+18,255,255])
    print(a,b)

In [ ]:
#opencv读取图片，默认是BGR
img=cv2.imread('img/test.jpg',cv2.IMREAD_COLOR)
img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()